# Importing a model
In this short notebook, we will import the model that was serialized in `main.ipynb`.

To import the model we can use `ort.InferenceSession` to read the model.

To get the necessary inputs, we can call `get_inputs()` on that session.

In [8]:
import onnxruntime as ort
import numpy as np
import pandas as pd
from pathlib import Path

# Column definitions
# must match those used during training in main.ipynb
numerical_cols = [
    "age", "daily_gaming_hours", "sleep_hours", "weight_change_kg",
    "exercise_hours_weekly", "social_isolation_score",
    "face_to_face_social_hours_weekly", "monthly_game_spending_usd", "years_gaming"
]
categorical_cols = [
    "gender", "game_genre", "gaming_platform", "sleep_quality",
    "sleep_disruption_frequency", "academic_work_performance",
    "mood_state", "mood_swing_frequency"
]
boolean_features = [
    "withdrawal_symptoms", "loss_of_other_interests",
    "continued_despite_problems", "eye_strain", "back_neck_pain"
]

session = ort.InferenceSession("best_model.onnx")
print("Model loaded successfully.")
print("Expected inputs:", [i.name for i in session.get_inputs()])

Model loaded successfully.
Expected inputs: ['age', 'gender', 'daily_gaming_hours', 'game_genre', 'gaming_platform', 'sleep_hours', 'sleep_quality', 'sleep_disruption_frequency', 'academic_work_performance', 'mood_state', 'mood_swing_frequency', 'withdrawal_symptoms', 'loss_of_other_interests', 'continued_despite_problems', 'eye_strain', 'back_neck_pain', 'weight_change_kg', 'exercise_hours_weekly', 'social_isolation_score', 'face_to_face_social_hours_weekly', 'monthly_game_spending_usd', 'years_gaming']


Finally, for inference, we can do `session.run(None, input)`.

In this example, we will use existing data from the dataset we used to train, just so we have a real value to verify our predictions against.

In [9]:
import kagglehub

path = kagglehub.dataset_download("shaistashahid/gaming-and-mental-health")
data = pd.read_csv(Path(path) / "Gaming and Mental Health.csv")

# drop the same columns that were excluded during training
x = data.drop(columns=["gaming_addiction_risk_level", "record_id", "primary_game",
                        "grades_gpa", "work_productivity_score"])
y_true = data["gaming_addiction_risk_level"]

# each feature group needs its own dtype
sample = x.iloc[:5]
input_feed = (
    {col: sample[[col]].values.astype(np.float32) for col in numerical_cols} |
    {col: sample[[col]].values for col in categorical_cols} |
    {col: sample[[col]].values.astype(np.int64) for col in boolean_features}
)

predictions = session.run(None, input_feed)[0]

label_map = {0: "Low", 1: "Moderate", 2: "High", 3: "Severe"}
print("Predictions:", [label_map[p] for p in predictions])
print("Actual:     ", y_true.iloc[:5].tolist())

Predictions: ['Severe', 'Low', 'Severe', 'High', 'Moderate']
Actual:      ['Severe', 'Low', 'Severe', 'High', 'Moderate']
